In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy.stats import pearsonr

# Influence of smoothing on correlations

To illustrate the effect of smoothing on the null-distribution of correlation coefficients we can run a small simulation study:

We generate a number of white noise samples that we correlate to generate a baseline null-distribution that emulates what a classical t-test would test against. We than filter the same time series with increasing $sigma$ and recalculate the correlations to obtain null-distributions for the smoothed data.

In [ ]:
n_obs = 2000   # number of observations
n_rep = 10000  # number of repetitions

# Generate n_rep white noise time series with length n_obs
x = np.random.randn(n_rep, n_obs)
y = np.random.randn(n_rep, n_obs)

# Calculate correlation between pairs of unfiltered time series
# This will serve as our baseline
r_null_unfilted = np.array([pearsonr(xi, yi)[0] for xi, yi in zip(x, y)])

# Filter width we want to look at
sigma_filt = np.array((2, 5, 10, 20))
# Output array
r_null = np.zeros((len(sigma_filt), n_rep))

# Filter white noise with the different filter lengths and save correlation coefficients for later
for i, s in enumerate(sigma_filt):
    xf = gaussian_filter1d(np.random.randn(n_rep, n_obs), s)
    yf = gaussian_filter1d(np.random.randn(n_rep, n_obs), s)
    r_null[i] = np.array([pearsonr(xi, yi)[0] for xi, yi in zip(xf, yf)])

First, lets take a look at histograms for the simulations

In [ ]:
bins = np.linspace(-0.5, 0.5, 50)
plt.hist(r_null_unfilted, bins, histtype='step', density=True, color='k', label='unfilted')
for i, s in enumerate(sigma_filt):
    plt.hist(r_null[i], bins, histtype='step', density=True, label='$\sigma=%.1f$' % s)
    
plt.xlabel('$r$')
plt.ylabel('Probability density')
plt.legend()
plt.title('Null-distribution for $r$, gaussian smoothing')

We can use the empirical distributions to calculate the value for $r$ that is the 95th percentile in the unsmoothed data. This value needs to be exeeded for the correlation to be significant at the (1 - 0.95) = 0.05 significance level.

In [ ]:
r_95 = np.percentile(r_null_unfilted, 95)
print('Empircial r_crit for unsmoothed data: %.3f', r_95)

Again using the samples we just generated, we can check the fraction of correlations that exeeds this threshhold. This gives an indication of how often we would call a random correlation significant, if we were to use the threshold of the white noise hypothesis

In [ ]:
fpr = np.mean(np.abs(r_null) >= r_95, axis=1)
print('P(r>=r_95) for smoothed data:', fpr)

To illustrate the rapid increase in the false-positive rate we can plot these values against the filter widths:

In [ ]:
plt.plot(sigma_filt, np.mean(np.abs(r_null) > r_95, axis=1), 'k.-')
plt.xlabel('Width of gaussian filter ($\sigma$)')
plt.ylabel('False positive rate at $\\alpha=%.2f$' % (0.05))

# Example

To go back to the example from the lecture we load the data and plot it alongside the two smoothed versions

In [ ]:
d = pd.read_csv('smoothing_example_data.csv', index_col=0)

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(12, 5))

sigma_f = 10

for i, (ax, c) in enumerate(zip(axes, d.columns)):
    ax.plot(d.index, d[c], '0.3', lw=0.75, label='annual')
    ax.plot(d.index, gaussian_filter1d(d[c], sigma_f), 'r', lw=2.0, label='%u yr gaussian' % sigma_f)
    ax.set_ylabel('Anomaly (°C)')
    ax.set_xlim(0, 2000)
    ax.text(0.01, 0.95, 'Site %u' %(i+1), va='top', ha='left', transform=ax.transAxes, color='k', fontsize=12)
    
axes[1].legend(loc='upper center', ncol=2)
axes[-1].set_xlabel('Year (C.E.)')


print('Annual: r=%.3f, (p=%.6f)' % pearsonr(d['y1'], d['y2']))
print('%u yr : r=%.3f, (p=%.6f)' % (sigma_f, *pearsonr(gaussian_filter1d(d['y1'], sigma_f), 
                                                 gaussian_filter1d(d['y2'], sigma_f))))

# Save correlation value for later
corr = pearsonr(gaussian_filter1d(d['y1'], sigma_f), 
                gaussian_filter1d(d['y2'], sigma_f))[0]

To generate a more relistic null-distribution we use again AR(1) processes. 

In [ ]:
from ar1 import fit_ar1, sample_ar1

First we estimate the auto-correlation as well as the standard deviations of the AR(1) processes from the data.

We see that both time series have very high auto-correlation.

In [ ]:
ar_y1 = fit_ar1(d.iloc[:,0].values - d.iloc[:,0].values.mean())
ar_y2 = fit_ar1(d.iloc[:,1].values - d.iloc[:,0].values.mean())

print('y1 (phi, sigma_e):', ar_y1)
print('y2 (phi, sigma_e):', ar_y2)

We than generate a large number of samples from AR(1) processes with these parametres that have the same number of observations as the original data. These we correlate with each other to generate the null-distribution we compare against.

In [ ]:
n_obs = d.shape[0]
n_samples = 10000

y1_samples = gaussian_filter1d(sample_ar1(n_obs, *ar_y1, n_samples), sigma_f)
y2_samples = gaussian_filter1d(sample_ar1(n_obs, *ar_y2, n_samples), sigma_f)
r_null_dist = np.array([pearsonr(y1i, y2i)[0] for y1i, y2i in zip(y1_samples, y2_samples)])

Lets plot the the null distribution together with the correlation of the smoothed data.

In [ ]:
fig, ax = plt.subplots()
ax.hist(r_null_dist, density=True, bins=25, range=(-0.8, 0.8), histtype='step', color='k')
ax.axvline(corr, lw=2.0, ls='dashed')
ax.set_xlabel('$r$')
ax.set_ylabel('Probability density')
ax.set_title('Empirical null distribution (smoothed AR(1))')

And lets calculate the empirical p-value of this correlation

In [ ]:
p_val = np.mean(np.abs(r_null_dist) >= corr)
print('r=%.2f, p=%.2f' % (corr, p_val))